In [1]:
using CSV
using DataFrames
using Serialization
using Dates

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/reported_hospital_capacity_admissions_facility_level_weekly_average_timeseries_20210228.csv"));

In [4]:
bad_ids = begin
    id_counts = combine(groupby(rawdata, :hospital_pk), :hospital_name => (x -> length(unique(x))) => :n_names)
    filter!(x -> x.n_names > 1, id_counts)
    unique(id_counts.hospital_pk)
end
filter!(row -> !(row.hospital_pk in bad_ids), rawdata);

In [5]:
findmissing(x) = (ismissing(x) || x == -999999) ? missing : x;

In [6]:
data_weekly = select(rawdata,
    :hospital_name => :hospital,
    :hospital_pk => :hospital_id,
    :collection_week => :date,
    
    :all_adult_hospital_inpatient_beds_7_day_avg => ByRow(findmissing) => :total_beds,
    :all_adult_hospital_inpatient_bed_occupied_7_day_avg => ByRow(findmissing) => :total_occupancy,
    
    :total_staffed_adult_icu_beds_7_day_avg => ByRow(findmissing) => :icu_beds,
    :staffed_adult_icu_bed_occupancy_7_day_avg => ByRow(findmissing) => :icu_occupancy,
);
sort!(data_weekly, [:hospital, :hospital_id, :date]);

In [7]:
function latest_val(xs)
    xs = filter(x -> !ismissing(x), xs)
    z = length(xs) == 0 ? missing : xs[end]
    z = coalesce(z, 0)
    return z
end;

In [8]:
data_latest = combine(groupby(data_weekly, [:hospital, :hospital_id]), [
    :total_beds => latest_val => :total_beds,
    :total_occupancy => latest_val => :total_occupancy,
    :icu_beds => latest_val => :icu_beds,
    :icu_occupancy => latest_val => :icu_occupancy,
]);

In [9]:
compute_load(a,b) = (a==0) ? 0.0 : (b==0) ? 1.0 : a/b;

In [10]:
insertcols!(data_latest, 5, :total_load => compute_load.(data_latest.total_occupancy, data_latest.total_beds));
insertcols!(data_latest, 8, :icu_load => compute_load.(data_latest.icu_occupancy, data_latest.icu_beds));

In [11]:
metadata = DataFrame(CSV.File("../data/hhs_hospital_meta.csv"));

In [12]:
hospital_locations_data = DataFrame(CSV.File("../data/hhs_hospital_locations.csv"))
select!(hospital_locations_data, :hospital_id, :lat, :long);

In [13]:
data_combined_all = leftjoin(data_latest, metadata, on=:hospital_id, makeunique=true);
data_combined_all = leftjoin(data_combined_all, hospital_locations_data, on=:hospital_id);

In [14]:
data_combined_all.lat = coalesce.(data_combined_all.lat, 0.0);
data_combined_all.long = coalesce.(data_combined_all.long, 0.0);

In [15]:
data_combined = select(data_combined_all,
    :hospitalname => :hospital,
    :hospital => :hhsname,
    :hospital_id,
    :lat,
    :long,
    :total_beds,
    :total_occupancy,
    :total_load,
    :icu_beds,
    :icu_occupancy,
    :icu_load,
)

,hospital,hhsname,hospital_id,lat,long,total_beds,total_occupancy,total_load,icu_beds,icu_occupancy,icu_load
,String?,String,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Abbeville Area Medical Center,ABBEVILLE AREA MEDICAL CENTER,421301,34.1579,-82.3815,18.0,11.9,0.661111,0.0,0.0,0.0
2,Abbeville General Hospital,ABBEVILLE GENERAL HOSPITAL,190034,29.9737,-92.1077,40.1,24.0,0.598504,6.1,5.0,0.819672
3,Abbott Northwestern Hospital,ABBOTT NORTHWESTERN HOSPITAL,240057,44.9526,-93.262,436.7,403.3,0.923517,46.9,46.9,1.0
4,Abilene Regional Medical Center,ABILENE REGIONAL MEDICAL CENTER,450558,32.3747,-99.744,99.9,92.9,0.92993,16.1,16.1,1.0
5,Abington Memorial Hospital,ABINGTON MEMORIAL HOSPITAL,390231,40.1191,-75.1199,524.9,461.4,0.879025,56.3,57.7,1.02487
6,Abraham Lincoln Memorial Hospital,ABRAHAM LINCOLN MEMORIAL HOSPITAL,141322,40.1535,-89.39,25.0,13.6,0.544,0.0,0.0,0.0
7,Abrazo Arrowhead Campus,ABRAZO ARROWHEAD CAMPUS,030094,33.6544,-112.202,196.6,196.6,1.0,37.1,28.7,0.773585
8,Abrazo Central Campus,ABRAZO CENTRAL CAMPUS,030030,33.5256,-112.102,125.3,125.3,1.0,27.0,23.1,0.855556
9,Abrazo Scottsdale Campus,ABRAZO SCOTTSDALE CAMPUS,030083,33.6393,-111.998,60.6,60.6,1.0,14.0,8.4,0.6


In [16]:
data_combined |> CSV.write("../data/hhs_current_load.csv");

In [17]:
data_combined_list = collect([NamedTuple(h) for h in eachrow(data_combined)]);

In [18]:
serialize("../data/hhs_current_load.jlser", data_combined_list)